In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_company_cdp_score(company_ticker):
    url = f"https://www.google.com/finance/quote/{company_ticker}:ETR"

    # Headers to mimic a browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        target_element = soup.find(string="CDP Climate Change Score")
        
        if target_element:
            parent = target_element.find_parent("span").find_parent("div")
            grade_element = parent.find("div", recursive=False)
            return grade_element.text
    return None


esg_documents_df = pd.read_csv('./dax_esg_media_dataset/esg_documents_for_dax_companies.csv', sep='|')
for index, symbol in enumerate(esg_documents_df['symbol']):
    print(f"Row {index}, Ticker: {symbol}, score: {get_company_cdp_score(symbol)}")

Row 0, Ticker: BEI, score: A
Row 1, Ticker: DTE, score: A
Row 2, Ticker: VNA, score: A-
Row 3, Ticker: MRK, score: A-
Row 4, Ticker: MTX, score: B
Row 5, Ticker: EOAN, score: A
Row 6, Ticker: RWE, score: A-
Row 7, Ticker: HEI, score: A
Row 8, Ticker: HEI, score: A
Row 9, Ticker: SIE, score: A
Row 10, Ticker: LIN, score: None
Row 11, Ticker: RWE, score: A-
Row 12, Ticker: QIA, score: B
Row 13, Ticker: HNK, score: None
Row 14, Ticker: LIN, score: None
Row 15, Ticker: DAI, score: None
Row 16, Ticker: nan, score: None
Row 17, Ticker: EOAN, score: A
Row 18, Ticker: HNK, score: None
Row 19, Ticker: CON, score: A
Row 20, Ticker: BAYN, score: A-
Row 21, Ticker: DTE, score: A
Row 22, Ticker: VOW3, score: A-
Row 23, Ticker: FRE, score: B
Row 24, Ticker: SY1, score: A
Row 25, Ticker: SRT, score: C
Row 26, Ticker: PAH3, score: A-
Row 27, Ticker: SAP, score: B
Row 28, Ticker: ADS, score: A-
Row 29, Ticker: DBK, score: B
Row 30, Ticker: PUM, score: A
Row 31, Ticker: PAH3, score: A-
Row 32, Ticker: P

KeyboardInterrupt: 